In [56]:
import os

In [57]:
%pwd

'C:\\Users\\2021ICTS28\\Desktop\\Kidney_disease_classification_using_MLFlow-DVC'

In [58]:
os.chdir('C:\\Users\\2021ICTS28\\Desktop\\Kidney_disease_classification_using_MLFlow-DVC')
%pwd

'C:\\Users\\2021ICTS28\\Desktop\\Kidney_disease_classification_using_MLFlow-DVC'

In [59]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    sourse_url: str
    local_data_file: Path
    unzip_dir: Path

In [60]:
from kidneyProject.constants import *
from kidneyProject.utils.common import read_yaml, create_directories

In [61]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config['artifacts_root']])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            sourse_url = config.sourse_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [62]:
import os
import zipfile
import gdown
from kidneyProject import logger
from kidneyProject.utils.common import get_size

In [63]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_data(self) -> str:
        
        try:
            dataset_url = self.config.sourse_url
            zip_download_path = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True) 
            logger.info(f"Downloading file from :[{dataset_url}] to :[{zip_download_path}]")
            
            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_path)
            
            logger.info(f"File downloaded successfully and saved at :[{zip_download_path}]")
        
        except Exception as e:
            raise e
        
        
    def extract_zip_file(self) -> None:
        
        unzip_path = Path(self.config.unzip_dir)
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            logger.info(f"Extracting zip file :[{self.config.local_data_file}] to :[{unzip_path}]")
            zip_ref.extractall(unzip_path)
            logger.info(f"File extracted successfully at :[{unzip_path}] and size of data is :[{get_size(unzip_path)}]")
            
            

In [64]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2026-01-13 11:06:36,892: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-13 11:06:36,892: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-13 11:06:36,892: INFO: common: created directory at: artifacts]
[2026-01-13 11:06:36,892: INFO: common: created directory at: artifacts/data_ingestion]
[2026-01-13 11:06:36,892: INFO: 3505312728: Downloading file from :[https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing] to :[artifacts/data_ingestion/data.zip]]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=015b9b72-46ee-4726-b973-f7acf349bfce
To: C:\Users\2021ICTS28\Desktop\Kidney_disease_classification_using_MLFlow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:06<00:00, 9.61MB/s]

[2026-01-13 11:06:45,878: INFO: 3505312728: File downloaded successfully and saved at :[artifacts/data_ingestion/data.zip]]
[2026-01-13 11:06:45,898: INFO: 3505312728: Extracting zip file :[artifacts/data_ingestion/data.zip] to :[artifacts\data_ingestion]]


[2026-01-13 11:06:46,281: INFO: 3505312728: File extracted successfully at :[artifacts\data_ingestion] and size of data is :[~ 0 KB]]
